In [421]:
import numpy as np 
import pandas as pd
import regex as re
import unicodedata as ud
import nltk
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import wordpunct_tokenize
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
import multiprocessing
from tqdm import tqdm
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import numpy as np

## Movies Datasets

In [68]:
forties_df = pd.read_csv('1940s_data/1940s_movies.csv')
fifties_df = pd.read_csv('1950s_data/1950s_movies.csv')
sixties_df = pd.read_csv('1960s_data/1960s_movies.csv')
seventies_df = pd.read_csv('1970s_data/1970s_movies.csv')

#### EDA functions

In [4]:
def describe_dataframe(df):
    
    for column in df.columns:
        
        print('Column Name: ', u'\u202B' + str(column) + u'\u202C')
        print(df[column].describe())
        print('\n'+'-'*20)

#### Cleansing functions

In [5]:
def change_empty_entries_to_None(df):
    for column in df.columns:
        for row_index in range(df.shape[0]):
            if '??' in str(df.loc[row_index, column]) or \
            df.loc[row_index, column] == 'nan' or \
            df.loc[row_index, column] == '-' or \
            type(df.loc[row_index, column]) == float:
                    df.loc[row_index, column] = None
    return df
    

In [6]:
def unify_date_format(df):
    for row in range(df.shape[0]):
        
        release_day = df.iloc[row, 15]
        if len(str(release_day)) == 4:
            df.iloc[row,17] = release_day
            df.iloc[row,16] = 'يناير'

    return df

In [7]:
def expand_release_date(df):
    # line continuation operators  '\' used for cleaner code
    df[['اليوم', 'الشهر', 'السنة']] = \
    df['تاريخ العرض'].str.split(' ', expand = True)
    
    return df
    

In [8]:
def parse_line(line_to_parse):
    
            parsed_values = line_to_parse.split('||')
            movie_name_date = parsed_values[0].strip(')').split('(')
            
            name = movie_name_date[0]
            year = movie_name_date[1]
            df_feature = parsed_values[1]
            
            return name, year, df_feature

In [9]:
def add_scraped_features(df, file_path, column_name):
    
    with open(file_path, 'r') as filehandle:
        
        for line in filehandle:
            
            current_line = line[:-1]
            name, year, df_feature = parse_line(current_line)
            if(not(df_feature == '') \
              or (str(df_feature) == 'nan')):
                df.loc[(df['اسم الفيلم'] == name) \
                     & (df['السنة'] == year), column_name] = df_feature     
    return df

In [10]:
def filter_foregin_movies(df, file_path):
    
    with open(file_path, 'r') as filehandle:
            
        for line in filehandle:
            
            current_line = line[:-1]
            name, year, country = parse_line(current_line)
           
            if(not(('مصر') in country)):   
                movie_index = df.loc[(df['اسم الفيلم'] == name) \
                                     & (df['السنة'] == year)].index
                df.drop(inplace = True, index = movie_index)
    return df

In [11]:
def get_avg_short_movie_time(df):
    
    short_movies_durations = df[' مدة الفيلم (دقيقة)'].loc[ \
                                    df[' تصنيف الفيلم'] == 'ﻗﺼﻴﺮ']
    avg_duration = np.average(list(short_movies_durations))
    
    return avg_duration

In [12]:
def get_null_percentage(df):
    
    total_count = df.shape[0]
    for column in df.columns:
    
        missing_count = df[column].isna().sum()
        missing_percentage = (missing_count/total_count)*100
        print('>> ' + u'\u202B' + column + ': '+ str(round(missing_percentage)) + '%')

In [13]:
actor_gender_dictionary = pd.read_csv('actor_gender_dictionary.csv')

In [14]:
actor_gender_dictionary.sample()

,Unnamed: 0,name,gender
480,481,سعاد فوزي,أنثى


In [15]:
actor_gender_dictionary.drop(columns = ['Unnamed: 0'], inplace= True)

In [16]:
actor_gender_dictionary.sample()

,name,gender
1230,وحيد جلال,ذكر


In [17]:
def seperate_cast_by_gender(movies_df, gender_df = actor_gender_dictionary):
    
    movies_df['ممثلات'] = None
    movies_df['ممثلون'] = None
    
    for row_index in range(movies_df.shape[0]):
        
        cast = movies_df.loc[row_index, 'تمثيل']
        try:
            actresses = []
            actors = []
            if( ',' in cast):
                
                cast = cast.split(',')
    
                for actor in cast:
             
                    gender = gender_df.loc[gender_df['name'] == actor]['gender']

                    if(gender.values == 'ذكر'):
                        actors.append(actor)
                        
                    if(gender.values == 'أنثى'):
                        actresses.append(actor)
                        
                movies_df.at[row_index, 'ممثلات'] = actresses
                movies_df.at[row_index, 'ممثلون'] = actors
                        
        except:
                'Unassigned: ' + str(actor) + ' in row: ' + str(row_index)
                
        finally:
                movies_df.at[row_index, 'ممثلات'] = actresses
                movies_df.at[row_index, 'ممثلون'] = actors
    return movies_df  

### *_1940s Movies_*

#### EDA

In [18]:
describe_dataframe(forties_df)

Column Name:  ‫اسم الفيلم‬
count             331
unique            330
top       خاتم سليمان
freq                2
Name: اسم الفيلم, dtype: object

--------------------
Column Name:  ‫تاريخ العرض‬
count      331
unique     216
top       1946
freq        24
Name: تاريخ العرض, dtype: object

--------------------
Column Name:  ‫التصنيف‬
count       127
unique        9
top       ??ا??
freq         46
Name: التصنيف, dtype: object

--------------------
Column Name:  ‫المدة‬
count    331.000000
mean     106.399698
std       11.097976
min       70.000000
25%      100.000000
50%      105.000000
75%      110.000000
max      152.000000
Name: المدة, dtype: float64

--------------------
Column Name:  ‫ملخص‬
count                                                   316
unique                                                  316
top       تدور قصة العمل حول الدكتور حلمي والذي يقع في ح...
freq                                                      1
Name: ملخص, dtype: object

--------------------
Column N

In [19]:
forties_df.loc[forties_df['اسم الفيلم'] == 'خاتم سليمان']

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
174,خاتم سليمان,1946,NaN,103.2,NaN,أبو بثينة,أمينة شريف,أفلام حرب,NaN,NaN,NaN,NaN,NaN,فؤاد شبل,NaN
227,خاتم سليمان,1947,ا????ا??,117.0,تناول الفيلم في إطار من الخيال، قصة المعلم بيو...,"حسن رمزي,السيد زيادة","ليلى مراد,يحيى شاهين,زكي رستم,نيللي مظلوم,ثريا...",أفلام النصر (محمد حسن رمزي وشركاه),"مصطفى حسن,وحيد فريد","ألبير نجيب,وفيقة أبو جبل",حبيب خوري,قاسم وجدي,"حسين السيد,السيد زيادة,رياض السنباطي,محمد القص...","حسن رمزي,ألبير نجيب,عبدالرحمن شريف",NaN


In [20]:
forties_df.dtypes

اسم الفيلم      object
تاريخ العرض     object
التصنيف         object
المدة          float64
ملخص            object
تأليف           object
تمثيل           object
إنتاج           object
تصوير           object
مونتاج          object
ديكور           object
ملابس           object
موسيقى          object
إخراج           object
توزيع           object
dtype: object

In [21]:
forties_df['التصنيف'].value_counts()

??ا??       46
???????     26
??????      25
ا????ا??    16
?????        7
????         2
?????ا?      2
تاريخي       2
???          1
Name: التصنيف, dtype: int64

In [22]:
get_null_percentage(forties_df)

>> ‫اسم الفيلم: 0%
>> ‫تاريخ العرض: 0%
>> ‫التصنيف: 62%
>> ‫المدة: 0%
>> ‫ملخص: 5%
>> ‫تأليف: 7%
>> ‫تمثيل: 1%
>> ‫إنتاج: 26%
>> ‫تصوير: 15%
>> ‫مونتاج: 28%
>> ‫ديكور: 39%
>> ‫ملابس: 94%
>> ‫موسيقى: 35%
>> ‫إخراج: 6%
>> ‫توزيع: 49%


#### Cleansing

In [23]:
forties_df = change_empty_entries_to_None(forties_df)

In [24]:
forties_df['التصنيف'].value_counts()

تاريخي    2
Name: التصنيف, dtype: int64

In [25]:
# forties_df = check_for_non_numeric_types(forties_df, 'المدة')

In [26]:
expanded_date_40s = forties_df.copy(deep = True)

In [27]:
expanded_date_40s = expand_release_date(forties_df)
expanded_date_40s = unify_date_format(expanded_date_40s)

In [28]:
expanded_date_40s = expanded_date_40s.drop(columns=['تاريخ العرض','اليوم'])
expanded_date_40s['التقييم'] = None

In [29]:
foreigns_filtered_40s  = expanded_date_40s.copy(deep = True)

In [30]:
foreigns_filtered_40s = filter_foregin_movies(expanded_date_40s \
                                                , '1940s_data/countries_1940s.txt')

In [31]:
genre_added_40s = foreigns_filtered_40s.copy(deep = True)

In [32]:
genre_added_40s = add_scraped_features(foreigns_filtered_40s \
                                          , '1940s_data/1940s_genres.txt' \
                                         , 'التصنيف')

In [33]:
genre_added_40s['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                75
ﻛﻮﻣﻴﺪﻱ               23
موسيقي / استعراضي    13
ﺭﻭﻣﺎﻧﺴﻲ              12
ﻋﺎﺋﻠﻲ                 2
ﺟﺮﻳﻤﺔ                 2
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          2
ﺣﺮﺏ                   1
ﺧﻴﺎﻝ                  1
ﻣﻐﺎﻣﺮاﺕ               1
Name: التصنيف, dtype: int64

In [34]:
ratings_added_40s = genre_added_40s.copy(deep = True)

In [35]:
ratings_added_40s = add_scraped_features(genre_added_40s \
                                          , '1940s_data/1940s_ratings.txt' \
                                         , 'التقييم')

In [36]:
synopsis_added_40s = ratings_added_40s.copy(deep = True)

In [37]:
synopsis_added_40s = add_scraped_features(ratings_added_40s \
                                          , '1940s_data/1940s_synopses.txt' \
                                         , 'ملخص')

In [38]:
synopsis_added_40s = synopsis_added_40s.reset_index(drop=True)

In [39]:
synopsis_added_40s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم
225,خاتم سليمان,ﺩﺭاﻣﺎ,117.0,يتناول الفيلم في إطار من الخيال، قصة المعلم بي...,"حسن رمزي,السيد زيادة","ليلى مراد,يحيى شاهين,زكي رستم,نيللي مظلوم,ثريا...",أفلام النصر (محمد حسن رمزي وشركاه),"مصطفى حسن,وحيد فريد","ألبير نجيب,وفيقة أبو جبل",حبيب خوري,قاسم وجدي,"حسين السيد,السيد زيادة,رياض السنباطي,محمد القص...","حسن رمزي,ألبير نجيب,عبدالرحمن شريف",None,يناير,1947,5.1


In [40]:
get_null_percentage(synopsis_added_40s)

>> ‫اسم الفيلم: 0%
>> ‫التصنيف: 60%
>> ‫المدة: 0%
>> ‫ملخص: 2%
>> ‫تأليف: 7%
>> ‫تمثيل: 1%
>> ‫إنتاج: 26%
>> ‫تصوير: 14%
>> ‫مونتاج: 28%
>> ‫ديكور: 39%
>> ‫ملابس: 94%
>> ‫موسيقى: 35%
>> ‫إخراج: 6%
>> ‫توزيع: 48%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التقييم: 5%


In [41]:
synopsis_added_40s['التقييم'] = synopsis_added_40s['التقييم'].astype(float)

In [42]:
cast_seperated_40s = seperate_cast_by_gender(synopsis_added_40s)

/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [43]:
cast_seperated_40s.drop(inplace = True, columns = ['تمثيل'])

In [44]:
cast_seperated_40s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم,ممثلات,ممثلون
313,شارع البهلوان,ﺩﺭاﻣﺎ,99.0,(سعيد) متزوج من (أمينة) ويغار عليها غيرة حمقاء...,"عبدالحليم مرسى,صلاح أبو سيف,علي الزرقاني","أفلام جبرائيل تلحمي,جبرائيل تلحمي,السعيد صادق","وديد سري,مصطفى حسن","صلاح عز الدين,اميل بحرى","ستوديو مصر,عثمان حسين,أنطوان بوليزويس,نجيب خوري",None,"فتحي قورة,محمد حسن الشجاعي","صلاح أبو سيف,إبراهيم المنياوى,محمود فريد,حمادة...",أفلام النيل (جبرائيل تلحمى),ديسمبر,1949,6.5,"[كاميليا, لولا صدقي, زينات صدقي, حسنة سليمان, ...","[كمال الشناوي, حسن فايق, إسماعيل يس, عبدالحميد..."


In [45]:
cast_seperated_40s.drop(columns = ['توزيع', 'ملابس', 'ديكور', 'مونتاج', \
                         'تصوير', 'موسيقى', 'إنتاج'], inplace = True)

In [46]:
movies_40s_df = cast_seperated_40s[['اسم الفيلم', 'الشهر', 'السنة', 'التصنيف',\
                                    'التقييم', 'المدة', 'ملخص', 'ممثلات', 'ممثلون',
                                    'تأليف', 'إخراج']]

In [47]:
movies_40s_df.sample()

,اسم الفيلم,الشهر,السنة,التصنيف,التقييم,المدة,ملخص,ممثلات,ممثلون,تأليف,إخراج
119,فيلم بين نارين,يناير,1945,None,NaN,105.7,None,[],[],None,يحيى نجاتي


### *_1950s Movies_*

In [70]:
fifties_df.sample(10)

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
168,طريق السعادة,16 مارس 1953,None,100.0,يقع الموظف الصغير (علي) في مشاكل، بسبب طموح اب...,"السيد زيادة,كامل الحفناوي","كمال الشناوي,ماجدة,زهرة العلا,فريد شوقي,شكري س...","ستوديو الأهرام (ستوديوهات الأهرام),أفلام الحرية",محمود نصر,"مارسيل صالح,أحمد إسماعيل",دنجل (عجمي عبدالرحمن),None,None,"كامل الحفناوي,إبراهيم السيد",None
32,ياسمين,11 ديسمبر 1950,None,122.0,رب أسرة لا يحب خلف البنات تنجب زوجته ابنة أخرى...,"أبو السعود الإبياري,أنور وجدي","أنور وجدي,مديحة يسري,زكي رستم,فيروز,عبدالعزيز ...",الأفلام المتحدة (أنور وجدي وشركاه),"عبدالحليم نصر,محمود بكر,حسن داهش","كمال الشيخ,عبدالمنعم توفيق,زينب عويس","أنطوان بوليزويس,عثمان حسين,نجيب خوري",None,"أبو السعود الإبياري,منير مراد,محمد البكار,محمو...","أنور وجدي,حسن الصيفي,منير مراد",عبد الفتاح هريدي
264,تحيا الرجالة,16 اكتوبر 1954,None,105.0,تدور الأحداث حول الشقيقتين (نوجة) و(بطة) اللتي...,"كامل الحفناوي,السيد بدير","شريفة ماهر,كارم محمود,سميحة توفيق,محمود شكوكو,...","ستوديو جيزة,إميل يزبك,إيهاب حسن","زكريا منصور,عبدالمنعم بهنسي,حسن خليل,عبدالحليم...","أحمد إسماعيل,فكري رستم,مارسيل صالح","روبرت شارفنبرج,عباس حلمي,عبدالباسط البنان,عبدا...",أحمد الكسار,"كارم محمود,عبدالغني الشيخ,كمال منصور,حسن أبو ع...",كامل الحفناوي,منتخبات بهنا فيلم
15,آه من الرجالة,2 يونيو 1950,None,110.0,ميمى وفيفى وسوسو، ثلاث فتيات رشيقات، تعرضن لتج...,"أبو السعود الإبياري,حلمي رفلة","محمد فوزي,مديحة يسري,سميحة توفيق,زينات صدقي,إس...",None,"برونو سالفي,زكريا منصور",إحسان فرغل,"حسن حسن,عباس حلمي,ستوديوهات (حدائق) شبرا,حسن ح...",None,محمد فوزي,"حلمي رفلة,عاطف سالم,سعد عرفة",لينشتنر
442,إسماعيل يس طرزان,25 مايو 1958,None,105.0,تدور الأحداث حول شخص ثري يتوفى تاركاً ثروة ضخم...,أبو السعود الإبياري,"إسماعيل يس,فيروز,عبدالسلام النابلسي,حسن فايق,إ...","ابراهيم والى,حسن الصيفي","اسعد عزيز,عادل عبدالعظيم,محمد عبدالعظيم","جلال مصطفى,مارسيل صالح",عبدالمنعم على,None,عطية شرارة,"نيازي مصطفى,محمد عمارة,محمود الصيفي,السعيد صادق",أفلام نهضة الشرق (بول مراديان)
54,أولاد الشوارع,19 فبراير 1951,None,115.0,تدور أحداث الفيلم حول الضابط فايز المتزوج وله ...,يوسف وهبي,"يوسف وهبي,مديحة يسري,زوزو ماضي,سراج منير,فاخر ...",ستوديو مصر,مصطفى حسن,ماري شماس,أنطوان بوليزويس,None,إبراهيم حجايج,يوسف وهبي,None
411,إسماعيل يس في الأسطول,11 نوفمبر 1957,None,95.0,تدور أحداث الفيلم حول شاب يقع في حب ابنة عمه و...,"حسن توفيق,السيد بدير","إسماعيل يس,زهرة العلا,أحمد رمزي,محمود المليجي,...","ممفيس فيلم,خليل دياب,جمال الليثي,رمسيس نجيب,ال...","عبدالعزيز فهمي,محمود فهمي,عبدالله ياقوت","عطية عبده,جميل عبدالعزيز,انجا وميلا","أنطوان بوليزويس,عثمان حسين",None,منير مراد,"فطين عبدالوهاب,عبدالعزيز جاد,صالح فوزي",الشركة العربية للسينما
89,ابن الحلال,26 نوفمبر 1951,None,108.6,محمود رجب يمتلك ألفى فدان، مما يجعل شقيقه حسن ...,"علي الزرقاني,سيف الدين شوكت","تحية كاريوكا,محمود المليجي,فاتن حمامة,محسن سرح...","ستوديوهات (حدائق) شبرا,ممفيس فيلم",فيرى فاركاش (فرانسوا فاركاش),كمال أبو العلا,"ولي الدين سامح,سيد شاهين",None,محمد عبدالمطلب,"سيف الدين شوكت,إلياس متى",شارل ليفشتز
299,كابتن مصر,28 فبراير 1955,None,105.0,تدور الأحداث حول شاب يعشق كرة القدم، ويسعى لأن...,كمال محمد,"محمد الكحلاوي,إسماعيل يس,زهرة العلا,هدى شمس ال...","كمال الكحلاوى,أفلام القبيلة","مصطفى حسن,احمد عطية","حسن حلمي (حسنوف),مارسيل صالح","عباس حلمي,بكر نصر",None,محمد الكحلاوي,"بهاء الدين شرف,يوسف سلامة,محمد مفتاح",أفلام القبيلة
3,شاطئ الغرام,20 فبراير 1950,None,120.0,عادل شاب ثري يحيا في ترف ويصاحب الراقصات، على ...,"هنري بركات,علي الزرقاني,يوسف عيسى","ليلى مراد,حسين صدقي,تحية كاريوكا,ميمي شكيب,محس...","عبدالحليم نصر,سينما ستوديومصر,روفائيل جبور,اتح...","حسن داهش,عبدالحليم نصر,عبدالله ياقوت","كمال الشيخ,فتحى قاسم,زينب عويس","أنطوان بوليزويس,عثمان حسين,نجيب خوري,مصطفى عبد...",None,"محمد القصبجي,مأمون الشناوي,صالح جودت,أحمد صدقي...","هنري بركات,يوسف معلوف,حسن داهش,سعد عرفة,علي وجدي",منتخبات بهنا فيلم


In [71]:
describe_dataframe(fifties_df)

Column Name:  ‫اسم الفيلم‬
count              534
unique             533
top       هل أقتل زوجى
freq                 2
Name: اسم الفيلم, dtype: object

--------------------
Column Name:  ‫تاريخ العرض‬
count      534
unique     368
top       1953
freq        14
Name: تاريخ العرض, dtype: object

--------------------
Column Name:  ‫التصنيف‬
count         10
unique         1
top       تاريخي
freq          10
Name: التصنيف, dtype: object

--------------------
Column Name:  ‫المدة‬
count    534.000000
mean     108.119663
std       13.207141
min       53.000000
25%      100.000000
50%      108.350000
75%      115.000000
max      170.000000
Name: المدة, dtype: float64

--------------------
Column Name:  ‫ملخص‬
count                                                   513
unique                                                  513
top        في صحراء السويس تقع أرملة لعوب تملك أحد المقا...
freq                                                      1
Name: ملخص, dtype: object

--------------------


In [ ]:
fifties_df = change_empty_entries_to_None(fifties_df)

In [74]:
fifties_df['التصنيف'].value_counts()

تاريخي    10
Name: التصنيف, dtype: int64

In [75]:
fifties_df[fifties_df.duplicated(subset = ['اسم الفيلم'])]

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
441,هل أقتل زوجى,5 مايو 1958,None,87.0,تبدأ أحداث الفيلم بجو هادئ مرح فى أسرة مكونة م...,محمد كامل حسن المحامي,"محسن سرحان,سميرة أحمد,توفيق الدقن,سهير البابلي...","وليم رزق,حسن موافى,فرغل البارودي,حسام الدين مصطفى","البير رياض,كليليو (ك. ستشفللي)",حسن حلمي (حسنوف),"أنطوان بوليزويس,عبدالمنعم على",None,None,"حسام الدين مصطفى,سيمون صالح,مصطفى سمهان,إبراهي...",None


In [76]:
fifties_df.loc[fifties_df['اسم الفيلم'] == 'هل أقتل زوجى']

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
378,هل أقتل زوجى,1956,None,108.1,None,None,None,None,None,None,None,None,None,None,None
441,هل أقتل زوجى,5 مايو 1958,None,87.0,تبدأ أحداث الفيلم بجو هادئ مرح فى أسرة مكونة م...,محمد كامل حسن المحامي,"محسن سرحان,سميرة أحمد,توفيق الدقن,سهير البابلي...","وليم رزق,حسن موافى,فرغل البارودي,حسام الدين مصطفى","البير رياض,كليليو (ك. ستشفللي)",حسن حلمي (حسنوف),"أنطوان بوليزويس,عبدالمنعم على",None,None,"حسام الدين مصطفى,سيمون صالح,مصطفى سمهان,إبراهي...",None


In [77]:
expanded_date_50s = fifties_df.copy(deep = True)

In [78]:
expanded_date_50s = expand_release_date(fifties_df)
expanded_date_50s = unify_date_format(expanded_date_50s)

In [79]:
expanded_date_50s.drop(columns=['تاريخ العرض','اليوم'], inplace = True)
expanded_date_50s['التقييم'] = None

In [80]:
foreign_filtered_50s = expanded_date_50s.copy(deep = True)

In [81]:
foreign_filtered_50s = filter_foregin_movies(expanded_date_50s
                                               , '1950s_data/countries_1950s.txt')

In [82]:
ratings_added_50s = foreign_filtered_50s.copy(deep = True)

In [83]:
ratings_added_50s = add_scraped_features(foreign_filtered_50s \
                                           , '1950s_data/1950s_ratings.txt'
                                           , 'التقييم')

In [84]:
genres_added_50s = ratings_added_50s.copy(deep = True)

In [85]:
genres_added_50s = add_scraped_features(ratings_added_50s \
                                           , '1950s_data/1950s_genres.txt'
                                           , 'التصنيف')

In [86]:
synopsis_added_50s = genres_added_50s.copy(deep = True)

In [87]:
synopsis_added_50s = add_scraped_features(genres_added_50s \
                                          , '1950s_data/1950s_synopses.txt' \
                                         , 'ملخص')

In [88]:
synopsis_added_50s['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                192
ﻛﻮﻣﻴﺪﻱ                60
ﺭﻭﻣﺎﻧﺴﻲ               44
موسيقي / استعراضي     31
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          14
تاريخي                 9
ﺟﺮﻳﻤﺔ                  8
ﺣﺮﻛﺔ                   6
ﻋﺎﺋﻠﻲ                  3
ﺧﻴﺎﻝ ﻋﻠﻤﻲ              3
ﺣﺮﺏ                    2
ﻏﻤﻮﺽ                   2
ﻣﻐﺎﻣﺮاﺕ                1
ﺭﻋﺐ                    1
ﺧﻴﺎﻝ                   1
Name: التصنيف, dtype: int64

In [89]:
synopsis_added_50s = synopsis_added_50s.reset_index(drop=True)

In [90]:
synopsis_added_50s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم
26,أيام شبابي,ﺟﺮﻳﻤﺔ,115.0,شاب ثري يعتدى على شرف فتاة ويتركها بعد أن أنجب...,"صالح جودت,جمال مدكور","كمال الشناوي,تحية كاريوكا,شادية,محمود المليجي,...",أرمان بوهالوفيتش,"عبدالعزيز فهمي,محمود فهمي",إحسان فرغل,"حسن حامد,حسين الشريف (حسين شريف),عباس حلمي,ستو...",None,"شارل فوسكلو,إيليا ايسائيا,شريف سرى,فتحي قورة,أ...","جمال مدكور,عبدالرؤوف الشافعي",شارل ليفشتز,نوفمبر,1950,6.2


In [91]:
get_null_percentage(synopsis_added_50s)

>> ‫اسم الفيلم: 0%
>> ‫التصنيف: 29%
>> ‫المدة: 0%
>> ‫ملخص: 1%
>> ‫تأليف: 3%
>> ‫تمثيل: 0%
>> ‫إنتاج: 12%
>> ‫تصوير: 9%
>> ‫مونتاج: 13%
>> ‫ديكور: 14%
>> ‫ملابس: 84%
>> ‫موسيقى: 30%
>> ‫إخراج: 1%
>> ‫توزيع: 54%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التقييم: 6%


In [92]:
synopsis_added_50s['التقييم'] = synopsis_added_50s['التقييم'].astype(float)

In [93]:
synopsis_added_50s = synopsis_added_50s.reset_index(drop = True)

In [94]:
cast_seperated_50s = seperate_cast_by_gender(synopsis_added_50s)

/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [95]:
cast_seperated_50s.drop(inplace = True, columns = ['تمثيل'])

In [96]:
cast_seperated_50s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم,ممثلات,ممثلون
367,القلب له أحكام,ﺭﻭﻣﺎﻧﺴﻲ,95.0,تقع الفتاة الفقيرة طالبة كلية الطب في حب زميله...,"السيد بدير,حسن توفيق,علي الزرقاني",حلمي حليم,"أحمد خورشيد,وديع مدور","فتحى قاسم,انجا وميلا",أنطوان بوليزويس,None,None,"حلمي حليم,طلبة رضوان",None,نوفمبر,1956,7.2,"[فاتن حمامة, زينات صدقي, زوزو نبيل, سميحة أيوب...","[أحمد رمزي, عبدالسلام النابلسي, عبدالفتاح القص..."


In [97]:
cast_seperated_50s.drop(columns = ['توزيع', 'ملابس', 'ديكور', 'مونتاج', \
                         'تصوير', 'موسيقى', 'إنتاج'], inplace = True)

In [98]:
movies_50s_df = cast_seperated_50s[['اسم الفيلم', 'الشهر', 'السنة', 'التصنيف',\
                                    'التقييم', 'المدة', 'ملخص', 'ممثلات', 'ممثلون',
                                    'تأليف', 'إخراج']]

In [99]:
movies_50s_df.sample()

,اسم الفيلم,الشهر,السنة,التصنيف,التقييم,المدة,ملخص,ممثلات,ممثلون,تأليف,إخراج
137,شم النسيم,نوفمبر,1952,None,5.8,100.0,يروي الفيلم مظاهر عيد شم النسيم وذلك من خلال ا...,"[سميرة أحمد, زينات صدقي, رفيعة الشال, برلنتي ع...","[محمود شكوكو, حسن فايق, محمود عزمي, محمد توفيق...",السيد بدير,"فرنيتشو,حمادة عبدالوهاب"


### *_1960s Movies_*

In [100]:
sixties_df.sample(5)

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
117,بلا عودة,31 يناير 1961,ﺟﺮﻳﻤﺔ,120.0,تقع الفتاة الفقيرة (مريم فخر الدين) في فخ عصاب...,"فتحي زكي,عبدالفتاح مصطفى,ريمون نصور","مريم فخر الدين,رشدي أباظة,زوزو ماضي,إستيفان رو...","ممفيس فيلم,كريكور تشاكماكجيان,سيد علي","مسعود عيسى,عبدالله ياقوت,مكرم سالم","عطية عبده,ميشيل الياس,كمال فهمي","ماهر عبدالنور,عجمي عبدالرحمن",أحمد الكسار,NaN,"ريمون نصور,فؤاد شافعي,اسحق هرمينا",أفلام الشرق الأوسط (وجدى عنايت)
514,فندق الأحلام,1968,NaN,100.0,صديقان، يعملان حارسان على إحدى الفيللات الفخم...,"أنور عبدالملك,فاروق عجرمة,شكرالله نعمه","دريد لحام,نهاد قلعي,ميشلين ضو,فريال كريم,عمر ذ...","ستوديو سيريا فيلم,أفلام إلياس حداد,شكرالله نعمه","محمد الرواس,غسان هارون",NaN,ستوديوهات هارون,NaN,"ميشيل طعمه,إلياس الرحباني,صباح,دريد لحام","ألبير نجيب,زكي صالح,يوسف شرف الدين",إلياس الحلو
32,وداعا يا حب,25 ابريل 1960,اﺳﺘﻌﺮاﺿﻲ,135.0,يعثر المطرب الشاب (شريف) على سلسلة ذهبية أمام ...,فتحي أبو الفضل,"مريم فخر الدين,محرم فؤاد,آمال فريد,نجوى فؤاد,خ...","ماجد فيلم,حسن موافى,وليم رزق","محمد عبدالعظيم,عادل عبدالعظيم,محمد زقلط,إبراهي...","عطية عبده,جميل عبدالعزيز,انجا وميلا","عبدالحميد السخاوي,عباس حلمي,عبدالمنعم على",NaN,"مأمون الشناوي,عطية شرارة,منير مراد,بليغ حمدي,م...","حسام الدين مصطفى,طلعت علام",أفلام الشرق الأوسط (وجدى عنايت)
26,وطني وحبي,14 ابريل 1960,NaN,120.0,وحيد ضابط مصرى يشغل مركزًا كبيرًا فى القيادة ا...,"فؤاد القصاص,محمد مصطفى سامي,حسين صدقي","حسين صدقي,عمر الحريري,حسن البارودي,صلاح منصور,...","فاروق المعتم,أفلام مصر الحديثة ""حسين صدقي""","عادل عبدالعظيم,محمد عبدالعظيم",أحمد إسماعيل,"روبرت شارفنبرج,عبدالمنعم على,ستوديو الأهرام (س...",NaN,NaN,حسين صدقي,NaN
295,المراهقان,20 ابريل 1964,ﻛﻮﻣﻴﺪﻱ,108.0,دراما اجتماعية كوميدية عن شخصين في الخمسين من ...,"سيف الدين شوكت,محمد أبو يوسف","سعاد حسني,يوسف فخر الدين,عبدالقادر حسين,عبدالع...","عبدالمسيح,حلمي رفلة",NaN,حسين عفيفي,أنطوان بوليزويس,NaN,فؤاد الظاهري,"سيف الدين شوكت,طلعت علام",NaN


In [101]:
describe_dataframe(sixties_df)

Column Name:  ‫اسم الفيلم‬
count         651
unique        646
top       المتمرد
freq            4
Name: اسم الفيلم, dtype: object

--------------------
Column Name:  ‫تاريخ العرض‬
count      651
unique     324
top       1968
freq        42
Name: تاريخ العرض, dtype: object

--------------------
Column Name:  ‫التصنيف‬
count       359
unique       15
top       ﺩﺭاﻣﺎ
freq        115
Name: التصنيف, dtype: object

--------------------
Column Name:  ‫المدة‬
count    651.000000
mean     102.518280
std       13.958636
min       30.000000
25%       95.000000
50%      101.000000
75%      105.400000
max      195.000000
Name: المدة, dtype: float64

--------------------
Column Name:  ‫ملخص‬
count                                                   560
unique                                                  557
top       عام 1867 فى تونس، فى حى المزاريق، ينهب رجال ال...
freq                                                      4
Name: ملخص, dtype: object

--------------------
Column Name:  ‫تأليف‬
co

In [102]:
sixties_df['التصنيف'].value_counts()

ﺩﺭاﻣﺎ           115
ﺭﻭﻣﺎﻧﺴﻲ          87
ﻛﻮﻣﻴﺪﻱ           76
ﺟﺮﻳﻤﺔ            21
اﺳﺘﻌﺮاﺿﻲ         18
ﺣﺮﻛﺔ             11
تاريخي            9
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ      6
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ        5
ﻏﻤﻮﺽ              3
ﻣﻐﺎﻣﺮاﺕ           3
ﺧﻴﺎﻝ ﻋﻠﻤﻲ         2
ﻋﺎﺋﻠﻲ             1
ﻭﺛﺎﺋﻘﻲ            1
ﻗﺼﻴﺮ              1
Name: التصنيف, dtype: int64

In [103]:
sixties_df.dtypes

اسم الفيلم      object
تاريخ العرض     object
التصنيف         object
المدة          float64
ملخص            object
تأليف           object
تمثيل           object
إنتاج           object
تصوير           object
مونتاج          object
ديكور           object
ملابس           object
موسيقى          object
إخراج           object
توزيع           object
dtype: object

In [104]:
sixties_df[sixties_df.duplicated(subset = ['اسم الفيلم', 'تمثيل'])]

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
518,المتمرد,1968,NaN,100.0,عام 1867 فى تونس، فى حى المزاريق، ينهب رجال ال...,"محمد المرزوقى,عمار الخليفي","سميرة أحمد,طاهر وهب,حطاب الذيب,حسيبة رشدي,الطا...","تونس,عمار الخليفي",عز الدين بن عمار,"فوزية طابع,عمار الخليفي",محمد السهيلى,NaN,NaN,"مصطفى بن إبراهيم,عمار الخليفي",NaN
525,المتمرد,1968,NaN,100.0,عام 1867 فى تونس، فى حى المزاريق، ينهب رجال ال...,"محمد المرزوقى,عمار الخليفي","سميرة أحمد,طاهر وهب,حطاب الذيب,حسيبة رشدي,الطا...","تونس,عمار الخليفي",عز الدين بن عمار,"فوزية طابع,عمار الخليفي",محمد السهيلى,NaN,NaN,"مصطفى بن إبراهيم,عمار الخليفي",NaN
532,المتمرد,1968,NaN,100.0,عام 1867 فى تونس، فى حى المزاريق، ينهب رجال ال...,"محمد المرزوقى,عمار الخليفي","سميرة أحمد,طاهر وهب,حطاب الذيب,حسيبة رشدي,الطا...","تونس,عمار الخليفي",عز الدين بن عمار,"فوزية طابع,عمار الخليفي",محمد السهيلى,NaN,NaN,"مصطفى بن إبراهيم,عمار الخليفي",NaN


_**Cleansing**_

In [105]:
sixties_df = change_empty_entries_to_None(sixties_df)

In [106]:
sixties_df.drop_duplicates(subset = ['اسم الفيلم', 'تمثيل']
                          , inplace= True, keep = 'last')

In [107]:
sixties_df.drop(inplace = True, index = \
               sixties_df.loc[sixties_df['التصنيف'] == 'ﻭﺛﺎﺋﻘﻲ'].index)

In [108]:
sixties_df = sixties_df.reset_index(drop = True)

In [109]:
date_expanded_60s = sixties_df.copy(deep = True)

In [110]:
date_expanded_60s = expand_release_date(sixties_df)
date_expanded_60s = unify_date_format(date_expanded_60s)

In [111]:
date_expanded_60s.drop(columns=['تاريخ العرض', 'اليوم'], inplace= True)

In [112]:
# sixties_df.to_csv('1960s_data/1960s_no_duplicates.csv')

In [113]:
date_expanded_60s['التقييم'] = None

In [114]:
# date_expanded_1960s_df.to_csv('1960s_data/date_expanded_1960s.csv')

In [115]:
date_expanded_60s['التقييم'] = date_expanded_60s['التقييم'].astype(float)

_**Filling Missing Values**_

* **Add Webscraped Ratings**

In [116]:
foreigns_filtered_60s = date_expanded_60s.copy(deep = True)

In [117]:
foreigns_filtered_60s = filter_foregin_movies(date_expanded_60s \
                                                , '1960s_data/1960s_countries.txt')

In [118]:
ratings_added_60s = foreigns_filtered_60s.copy(deep = True)

In [119]:
ratings_added_60s = add_scraped_features(foreigns_filtered_60s \
                                    , '1960s_data/1960s_ratings.txt' \
                                    , 'التقييم')

In [120]:
genre_added_60s = ratings_added_60s.copy(deep = True)

In [121]:
# ratings_added_60s_df.to_csv('1960s_data/ratings_added_df_60s.csv')

In [122]:
genre_added_60s = add_scraped_features(ratings_added_60s \
                                          , '1960s_data/1960s_genres.txt' \
                                         , 'التصنيف')

In [123]:
# genre_added_60s_df.to_csv('1960s_data/genre_added_60s_df.csv')

In [124]:
genre_added_60s['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                171
ﻛﻮﻣﻴﺪﻱ                59
ﺭﻭﻣﺎﻧﺴﻲ               46
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          18
موسيقي / استعراضي     13
ﺟﺮﻳﻤﺔ                 12
ﺣﺮﻛﺔ                   7
ﻣﻐﺎﻣﺮاﺕ                4
ﺧﻴﺎﻝ                   4
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ             4
ﻋﺎﺋﻠﻲ                  4
ﺧﻴﺎﻝ ﻋﻠﻤﻲ              2
تاريخي                 2
ﻗﺼﻴﺮ                   1
ﻏﻤﻮﺽ                   1
Name: التصنيف, dtype: int64

In [125]:
synopsis_added_60s = genre_added_60s.copy(deep = True)

In [126]:
synopsis_added_60s = add_scraped_features(genre_added_60s \
                                          , '1960s_data/1960s_synopses.txt' \
                                         , genre_added_60s.columns[3])

In [127]:
synopsis_added_60s = synopsis_added_60s.reset_index(drop=True)

In [128]:
cast_seperated_60s = seperate_cast_by_gender(synopsis_added_60s)

cast_seperated_60s.drop(inplace = True, columns = ['تمثيل'])
cast_seperated_60s.sample()

/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم,ممثلات,ممثلون
81,لماذا أعيش,ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ,90.0,مصطفى شاب من أصل طيب، يتولى رعاية أسرته الفقير...,عبدالغني قمر,"ابراهيم والى,عاطف رزق,كمال صلاح الدين",محمود فهمي,مصطفى مختار,حلمي عزب,None,فؤاد الظاهري,"إبراهيم عمارة,محمد عمارة,بندق",أفلام إدوار خياط (ا. خيّاط),يناير,1961,7.6,"[سعاد حسني, زوزو نبيل, نجوى فؤاد, كوثر رمزي]","[شكري سرحان, محسن سرحان, كمال صلاح الدين, عبدا..."


In [129]:
get_null_percentage(cast_seperated_60s)

>> ‫اسم الفيلم: 0%
>> ‫التصنيف: 35%
>> ‫المدة: 0%
>> ‫ملخص: 7%
>> ‫تأليف: 9%
>> ‫إنتاج: 21%
>> ‫تصوير: 15%
>> ‫مونتاج: 23%
>> ‫ديكور: 30%
>> ‫ملابس: 86%
>> ‫موسيقى: 45%
>> ‫إخراج: 6%
>> ‫توزيع: 52%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التقييم: 10%
>> ‫ممثلات: 0%
>> ‫ممثلون: 0%


In [130]:
cast_seperated_60s.drop(columns = ['توزيع', 'ملابس', 'ديكور', 'مونتاج', \
                         'تصوير', 'موسيقى', 'إنتاج'], inplace = True)

In [131]:
movies_60s_df = cast_seperated_60s[['اسم الفيلم', 'الشهر', 'السنة', 'التصنيف',\
                                    'التقييم', 'المدة', 'ملخص', 'ممثلات', 'ممثلون',
                                    'تأليف', 'إخراج']]

In [132]:
movies_60s_df.sample()

,اسم الفيلم,الشهر,السنة,التصنيف,التقييم,المدة,ملخص,ممثلات,ممثلون,تأليف,إخراج
171,دنيا البنات,يناير,1962,None,5.9,88.0,أسرة مكونه من تسعة أبناء،الأب موظفٌ بسيط تزوجت...,"[ماجدة, شويكار, زينات صدقي, زيزي مصطفى, سلوى س...","[رشدي أباظة, محمد سلطان, سعيد أبو بكر, عبدالخا...",سعد عرفة,"سعد عرفة,سيمون صالح,عبدالمجيد سالم"


### *_1970s Movies_*

_**EDA**_

In [133]:
describe_dataframe(seventies_df)

Column Name:  ‫اسم الفيلم‬
count                       793
unique                      781
top       الحسناء و قاهر الفضاء
freq                          4
Name: اسم الفيلم, dtype: object

--------------------
Column Name:  ‫تاريخ العرض‬
count      793
unique     285
top       1972
freq        53
Name: تاريخ العرض, dtype: object

--------------------
Column Name:  ‫التصنيف‬
count       341
unique       18
top       ﺩﺭاﻣﺎ
freq        140
Name: التصنيف, dtype: object

--------------------
Column Name:  ‫المدة‬
count     793
unique     89
top       100
freq       93
Name: المدة, dtype: object

--------------------
Column Name:  ‫ملخص‬
count                                                   649
unique                                                  649
top       تطارد الشرطة سبع الليل تاجر المخدرات، فيقوم بإ...
freq                                                      1
Name: ملخص, dtype: object

--------------------
Column Name:  ‫تأليف‬
count          684
unique         491
top       فيص

In [134]:
seventies_df['ملخص'].isnull().sum()

144

In [135]:
seventies_df['التصنيف'].value_counts()

ﺩﺭاﻣﺎ           140
ﺭﻭﻣﺎﻧﺴﻲ          60
ﻛﻮﻣﻴﺪﻱ           56
ﺟﺮﻳﻤﺔ            22
اﺳﺘﻌﺮاﺿﻲ         15
تاريخي            8
ﺣﺮﻛﺔ              8
ﻗﺼﻴﺮ              5
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ      5
ﻣﻐﺎﻣﺮاﺕ           4
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ        4
ﻭﺛﺎﺋﻘﻲ            3
ﻏﻤﻮﺽ              3
ﺣﺮﺏ               3
ﺭﻳﺎﺿﻲ             2
ﺭﺳﻮﻡ ﻣﺘﺤﺮﻛﺔ       1
ﺧﻴﺎﻝ ﻋﻠﻤﻲ         1
ﻋﺎﺋﻠﻲ             1
Name: التصنيف, dtype: int64

In [136]:
seventies_df[seventies_df.duplicated(subset = ['اسم الفيلم', 'تمثيل'])]

,اسم الفيلم,تاريخ العرض,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
154,فرح زهران,1972,NaN,100.5,NaN,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
164,فرح زهران,1972,NaN,100.5,NaN,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
172,فرح زهران,1972,NaN,100.5,NaN,محسن زايد,"عبدالعزيز مخيون,إنعام الجريتلي,ممدوح زايد",NaN,NaN,NaN,NaN,NaN,NaN,محسن زايد,NaN
306,مخطوبتين بالوراثة,1974,NaN,108.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,الحسناء و قاهر الفضاء,1975,NaN,95,NaN,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN
414,الحسناء و قاهر الفضاء,1975,NaN,95,NaN,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN
421,الحسناء و قاهر الفضاء,1975,NaN,95,NaN,"محمد جومر,سهيل كنعان","محمود جبر,إغراء,أديب قدورة,نجاح حفيظ,أنور البا...","سميراميس فيلم,دمشق",محمد الرواس,مروان عكاوي,NaN,NaN,هنري,سهيل كنعان,NaN


In [137]:
seventies_df = change_empty_entries_to_None(seventies_df)

_**Cleansing**_

In [138]:
seventies_df.drop_duplicates(subset = ['اسم الفيلم', 'تمثيل'] \
                             , inplace= True, keep = 'last')

In [139]:
seventies_df = seventies_df.reset_index(drop = True)

_**Filling Missing Values**_

* **Add Webscraped Ratings**

In [140]:
date_expanded_70s = seventies_df.copy(deep = True)

In [141]:
date_expanded_70s = expand_release_date(seventies_df)
date_expanded_70s = unify_date_format(date_expanded_70s)

In [142]:
date_expanded_70s = date_expanded_70s.drop(columns=['تاريخ العرض', 'اليوم'])
date_expanded_70s['التقييم'] = None

In [143]:
foreign_filtered_70s = date_expanded_70s.copy(deep = True)

In [144]:
foreign_filtered_70s = \
    filter_foregin_movies(date_expanded_70s, '1970s_data/1970s_countries.txt')

In [145]:
ratings_added_70s = foreign_filtered_70s.copy(deep = True)

In [146]:
ratings_added_70s = add_scraped_features(foreign_filtered_70s \
                                    , '1970s_data/1970s_ratings.txt' \
                                    , 'التقييم')

In [147]:
genre_added_70s = ratings_added_70s.copy(deep = True)

In [148]:
genre_added_70s = add_scraped_features(ratings_added_70s \
                                          , '1970s_data/1970s_genres.txt'
                                         ,'التصنيف')

In [149]:
synopsis_added_70s = genre_added_70s.copy(deep = True)

In [150]:
synopsis_added_70s = add_scraped_features(genre_added_70s \
                                          , '1970s_data/1970s_synopses.txt' \
                                         , 'ملخص')

In [151]:
# foreigns_filtered_df_70s.to_csv('1970s_data/foreigns_filtered_70s.csv')

In [152]:
synopsis_added_70s = synopsis_added_70s.reset_index(drop=True)

In [153]:
synopsis_added_70s['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                192
ﻛﻮﻣﻴﺪﻱ                47
ﺭﻭﻣﺎﻧﺴﻲ               27
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          21
موسيقي / استعراضي     16
ﺟﺮﻳﻤﺔ                 12
ﺣﺮﻛﺔ                   6
ﻣﻐﺎﻣﺮاﺕ                4
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ             3
ﻏﻤﻮﺽ                   3
تاريخي                 3
ﻭﺛﺎﺋﻘﻲ                 2
ﻗﺼﻴﺮ                   2
ﺧﻴﺎﻝ                   1
ﻋﺎﺋﻠﻲ                  1
Name: التصنيف, dtype: int64

In [154]:
synopsis_added_70s.drop(inplace = True, index = \
            synopsis_added_70s.loc[synopsis_added_70s['التصنيف'] == 'ﻭﺛﺎﺋﻘﻲ'].index)

In [155]:
synopsis_added_70s = synopsis_added_70s.reset_index(drop =True)
synopsis_added_70s  = change_empty_entries_to_None(synopsis_added_70s)

In [156]:
synopsis_added_70s = synopsis_added_70s.reset_index(drop=True)

In [157]:
synopsis_added_70s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم
66,صبيحة,None,97.0,تهرب صبيحة من بلدتها بعدما تعرضت لاعتداء من أح...,"يوسف وهبي,نبيل غلام","ناهد يسري,جلال الشرقاوي,يوسف وهبي",None,None,None,None,None,None,حسن حافظ,None,يناير,1971,0


In [158]:
synopsis_added_70s['التقييم'] = synopsis_added_70s['التقييم'].astype(float)

In [159]:
synopsis_added_70s = change_empty_entries_to_None(synopsis_added_70s)

In [160]:
cast_seperated_70s = seperate_cast_by_gender(synopsis_added_70s)

cast_seperated_70s.drop(inplace = True, columns = ['تمثيل'])
cast_seperated_70s.sample()

/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/karefsn/Development/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع,الشهر,السنة,التقييم,ممثلات,ممثلون
141,مكان للحب,ﺩﺭاﻣﺎ,120,تقرر سعاد أن تنتقل من مدينة السويس إلى مدينة ا...,"مصطفى كامل,سعيد مرزوق",None,None,None,None,None,None,سعيد مرزوق,None,يوليو,1972,4.3,"[فيفي عبده, زيزي مصطفى, سعاد حسني]","[أسامة عباس, عادل نصيف, حسن حسين, أحمد أباظة, ..."


In [161]:
cast_seperated_70s.drop(columns = ['توزيع', 'ملابس', 'ديكور', 'مونتاج', \
                         'تصوير', 'موسيقى', 'إنتاج'], inplace = True)

In [162]:
cast_seperated_70s.sample()

,اسم الفيلم,التصنيف,المدة,ملخص,تأليف,إخراج,الشهر,السنة,التقييم,ممثلات,ممثلون
346,الحب تحت المطر,None,120,يتعرف المخرج السينمائي عادل على مرزوق ويعرض عل...,"نجيب محفوظ,ممدوح الليثي","أحمد رمزي,صلاح نظمي,حسين كمال,حسن إبراهيم",اكتوبر,1975,5.9,"[ميرفت أمين, ماجدة الخطيب, حياة قنديل, منى جبر...","[أحمد رمزي, عادل أدهم, عماد حمدي, محمود قابيل,..."


In [163]:
movies_70s_df = cast_seperated_70s[['اسم الفيلم', 'الشهر', 'السنة', 'التصنيف',\
                                    'التقييم', 'المدة', 'ملخص', 'ممثلات', 'ممثلون',
                                    'تأليف', 'إخراج']]

In [164]:
movies_70s_df.sample()

,اسم الفيلم,الشهر,السنة,التصنيف,التقييم,المدة,ملخص,ممثلات,ممثلون,تأليف,إخراج
423,سري جداً,يناير,1977,None,4.8,73,يتهم هشام بعملية اغتيال وتجسس، فيحاول ضابط الش...,"[نبيلة عبيد, عزة كمال, أميرة, بثينة نصار]","[سمير صبري, إبراهيم خان, محمود المليجي, صلاح ن...",سعيد عبدالغني,"عادل صادق,سمير علام,محمد محمود حسن,حسن حافظ"


### *_Merging  Datasets_*

In [165]:
dataframes = [movies_40s_df \
              , movies_50s_df\
              , movies_60s_df\
              , movies_70s_df]

In [166]:
movies_df = pd.concat(dataframes, sort = False)
movies_df = movies_df.reset_index(drop = True)

In [167]:
describe_dataframe(movies_df)

Column Name:  ‫اسم الفيلم‬
count     1980
unique    1956
top       دنيا
freq         2
Name: اسم الفيلم, dtype: object

--------------------
Column Name:  ‫الشهر‬
count      1980
unique       12
top       يناير
freq        710
Name: الشهر, dtype: object

--------------------
Column Name:  ‫السنة‬
count     1980
unique      40
top       1978
freq        78
Name: السنة, dtype: object

--------------------
Column Name:  ‫التصنيف‬
count      1195
unique       17
top       ﺩﺭاﻣﺎ
freq        630
Name: التصنيف, dtype: object

--------------------
Column Name:  ‫التقييم‬
count     1815.0
unique     105.0
top          0.0
freq        99.0
Name: التقييم, dtype: float64

--------------------
Column Name:  ‫المدة‬
count     1971.0
unique     179.0
top        100.0
freq       149.0
Name: المدة, dtype: float64

--------------------
Column Name:  ‫ملخص‬
count       1878
unique      1871
top             
freq           3
Name: ملخص, dtype: object

--------------------
Column Name:  ‫ممثلات‬
count     

In [168]:
get_null_percentage(movies_df)

>> ‫اسم الفيلم: 0%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التصنيف: 40%
>> ‫التقييم: 8%
>> ‫المدة: 0%
>> ‫ملخص: 5%
>> ‫ممثلات: 0%
>> ‫ممثلون: 0%
>> ‫تأليف: 7%
>> ‫إخراج: 5%


In [169]:
movies_df.dtypes

اسم الفيلم    object
الشهر         object
السنة         object
التصنيف       object
التقييم       object
المدة         object
ملخص          object
ممثلات        object
ممثلون        object
تأليف         object
إخراج         object
dtype: object

In [170]:
movies_df['المدة'] = movies_df['المدة'].astype(float)
movies_df['التقييم'] = movies_df['التقييم'].astype(float)

#### Further Cleansing

In [171]:
movies_df['المدة'] = movies_df['المدة'].fillna(float(0))
movies_df['التقييم'] = movies_df['التقييم'].fillna(float(0))

In [172]:
movies_df = movies_df.dropna(axis=0, subset=['ملخص'])

In [173]:
get_null_percentage(movies_df)

>> ‫اسم الفيلم: 0%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التصنيف: 37%
>> ‫التقييم: 0%
>> ‫المدة: 0%
>> ‫ملخص: 0%
>> ‫ممثلات: 0%
>> ‫ممثلون: 0%
>> ‫تأليف: 3%
>> ‫إخراج: 2%


## Doc2Vec on Synopsis  to Classify Movie Genre

#### Doc2Vec Preprocessing Functions

In [174]:
def remove_names(synopsis, actors, actresses):  
    
    if(type(actors) == list):
        for actor in actors:
            if(actor in synopsis):
                synopsis = synopsis.replace(actor, '')
    if(type(actresses) == list):
        for actress in actresses:
            if(actress in synopsis):
                synopsis = synopsis.replace(actress, '')
            
    return synopsis

In [175]:
def stem_synopsis(synopsis, stemmer):
    stemmed_synopsis = ''
    for word in synopsis.split():
        stemmed_word = stemmer.stem(word)
        if(stemmed_word != word):
            stemmed_synopsis += stemmed_word + ' '
        
    return stemmed_synopsis          

In [183]:
def remove_arabic_punctuation(synopsis):
    
    cleaned_synopsis = \
    ''.join(char for char in synopsis if not ud.category(char).startswith('P'))
    
    return cleaned_synopsis

In [184]:
def remove_stop_words(synopsis, stop_words):  

    for word in synopsis.split():
        if word in stop_words:
            synopsis = synopsis.replace(word, '')
            
    return synopsis

In [205]:
def remove_non_words(synopsis):
    synopsis = re.sub(r'(\d+)|[^\w\s]','',synopsis)
    synopsis = re.sub(r'\([^)]*\)', '', synopsis)
    return synopsis

In [186]:
def tokenize_synopsis(synopsis):
    tokens = []
    words = wordpunct_tokenize(synopsis)
    tokens.extend(words)
    return tokens

In [187]:
def preprocess_synopses(dataset):
    
    stop_words = set(nltk.corpus.stopwords.words("arabic"))
    stop_words.add('على')
    stop_words.add('ان')
    
    stemmer = ISRIStemmer()

    for row_index in range(dataset.shape[0]):
        
        synopsis = dataset.loc[row_index, 'ملخص']
        actors = dataset.loc[row_index, 'ممثلون']
        actresses = dataset.loc[row_index, 'ممثلات']
        synopsis = remove_names(synopsis, actors, actresses)
        synopsis = remove_arabic_punctuation(synopsis)
        synopsis = remove_non_words(synopsis)
        synopsis = remove_stop_words(synopsis, stop_words)
        stemmed_synopsis = stem_synopsis(synopsis, stemmer)
        tokenized_synopsis = tokenize_synopsis(stemmed_synopsis)
    
        try:
            dataset.at[row_index, 'ملخص'] = tokenized_synopsis
        except:
            print('error occured at row: ' + str(row_index))
                  
    return dataset

#### Create & Preprocess Dataset

In [393]:
movies_dataset = movies_df.copy(deep = True)
movies_dataset = movies_dataset.reset_index(drop = True)

In [394]:
movies_dataset = preprocess_synopses(movies_dataset)

In [395]:
targets = movies_dataset.loc[movies_dataset['التصنيف'].isnull()]

In [396]:
movies_dataset.dropna(subset=['التصنيف'], inplace=True)

In [397]:
movies_dataset = movies_dataset.reset_index(drop = True)

#### Reduce Genre Dimensionalites

In [398]:
movies_dataset['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                627
ﻛﻮﻣﻴﺪﻱ               189
ﺭﻭﻣﺎﻧﺴﻲ              129
موسيقي / استعراضي     72
ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ          55
ﺟﺮﻳﻤﺔ                 33
ﺣﺮﻛﺔ                  19
تاريخي                14
ﻋﺎﺋﻠﻲ                 10
ﻣﻐﺎﻣﺮاﺕ               10
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ             7
ﺧﻴﺎﻝ                   7
ﻏﻤﻮﺽ                   6
ﺧﻴﺎﻝ ﻋﻠﻤﻲ              5
ﺣﺮﺏ                    3
ﻗﺼﻴﺮ                   2
ﺭﻋﺐ                    1
Name: التصنيف, dtype: int64

We reduce the number of labels by grouping similar genres into one label,
* ﺩﺭاﻣﺎ/ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ/ﺣﺮﻛﺔ/ﺟﺮﻳﻤﺔ/ﻣﻐﺎﻣﺮاﺕ/ﻏﻤﻮﺽ = دراما
* ﺧﻴﺎﻝ ﻋﻠﻤﻲ/ﺧﻴﺎﻝ = ﺧﻴﺎﻝ
* ﻋﺎﺋﻠﻲ/ ﻛﻮﻣﻴﺪﻱ = ﻛﻮﻣﻴﺪﻱ

While the rest of the labels remain unchanged.


In [399]:
def reduce_genre_labels(ds):
    
    for row_index in range(ds.shape[0]):
        
        cell_value = ds.loc[row_index, 'التصنيف']
        
        if(cell_value == 'ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺩﺭاﻣﺎ'
        if(cell_value == 'ﺟﺮﻳﻤﺔ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺩﺭاﻣﺎ'
        if(cell_value == 'ﺣﺮﻛﺔ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺩﺭاﻣﺎ'
        if(cell_value == 'ﻏﻤﻮﺽ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺩﺭاﻣﺎ'
        if(cell_value == 'ﻣﻐﺎﻣﺮاﺕ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺩﺭاﻣﺎ'
        if(cell_value == 'ﻋﺎﺋﻠﻲ'):
            ds.loc[row_index, 'التصنيف'] = 'ﻛﻮﻣﻴﺪﻱ'
        if(cell_value == 'ﺧﻴﺎﻝ ﻋﻠﻤﻲ'):
            ds.loc[row_index, 'التصنيف'] = 'ﺧﻴﺎﻝ'

    return ds
        

In [400]:
movies_dataset = reduce_genre_labels(movies_dataset)

In [401]:
movies_dataset['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                750
ﻛﻮﻣﻴﺪﻱ               199
ﺭﻭﻣﺎﻧﺴﻲ              129
موسيقي / استعراضي     72
تاريخي                14
ﺧﻴﺎﻝ                  12
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ             7
ﺣﺮﺏ                    3
ﻗﺼﻴﺮ                   2
ﺭﻋﺐ                    1
Name: التصنيف, dtype: int64

Short movies cannot be predicted contextually, therefore we assign the genre "ﻗﺼﻴﺮ" to movies with duration less than 45 minutes, and remove them from our yet to be trained dataset.

In [402]:
def set_short_movies_genre(ds):
    for row_index in range(ds.shape[0]):
        cell_value = ds.loc[row_index, 'المدة']
        if(cell_value < float(45) and cell_value !=0):
            ds.loc[row_index, 'التصنيف'] = "ﻗﺼﻴﺮ"
            
    return ds

In [403]:
movies_dataset = set_short_movies_genre(movies_dataset)

In [404]:
movies_dataset['التصنيف'].value_counts()

ﺩﺭاﻣﺎ                748
ﻛﻮﻣﻴﺪﻱ               199
ﺭﻭﻣﺎﻧﺴﻲ              129
موسيقي / استعراضي     72
تاريخي                14
ﺧﻴﺎﻝ                  12
ﺳﻴﺮﺓ ﺫاﺗﻴﺔ             7
ﻗﺼﻴﺮ                   4
ﺣﺮﺏ                    3
ﺭﻋﺐ                    1
Name: التصنيف, dtype: int64

In [406]:
short_movies_indices = \
movies_dataset.loc[movies_dataset['التصنيف'] == 'ﻗﺼﻴﺮ'].index

In [407]:
short_movies = movies_dataset.take(short_movies_indices)

In [408]:
movies_dataset.drop(index = short_movies_indices,\
                                   inplace = True)

In [411]:
movies_dataset = movies_dataset.reset_index(drop = True)

#### Training and Testing 
Word2Vec Models used:
* Distributed Bag of Words (DBOW): 
        We achieve vectors for each synopsis by training a neural network.
        This neural network is tasked to predict the probability distribution
        of words in a snyopsis.
        
* Distributed Memory (DM):
        DM is a model that memorises what is missing from a synopsis context.

In [422]:
def fetch_targets_regressors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags, model.infer_vector(doc.words, steps=30)) for doc in sents])
    return targets, regressors

In [412]:
corpus = movies_dataset[['ملخص', 'التصنيف']]

In [414]:
train, test = \
train_test_split(corpus, test_size=0.3, random_state=42)

In [416]:
train_tagged = train.apply(
    lambda row: TaggedDocument(words=row['ملخص'], tags=row['التصنيف']), axis=1)
test_tagged = train.apply(
    lambda row: TaggedDocument(words=row['ملخص'], tags=row['التصنيف']), axis=1)

####  Training DBOW Model

In [419]:
cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([document for document in tqdm(train_tagged.values)])

100%|██████████| 829/829 [00:00<00:00, 718998.76it/s]


In [429]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([document for document in tqdm(train_tagged.values)]),\
                     total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 829/829 [00:00<00:00, 1417479.83it/s]


In [433]:
y_train, X_train = fetch_targets_regressors(model_dbow,\
                                            train_tagged)
y_test, X_test = fetch_targets_regressors(model_dbow,\
                                          test_tagged)
SGD_classifier = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
SGD_classifier.fit(X_train, y_train)
y_predict = SGD_classifier.predict(X_test)

print('Testing accuracy %s' % accuracy_score(y_test, y_predict))
print('Testing F1 score: {}'.format(f1_score(y_test, y_predict, average='weighted')))

Testing accuracy 0.6164053075995175
Testing F1 score: 0.47210099849058035


Which genres were never predicted?

In [434]:
set(y_test) - set(y_predict)

{'تاريخي', 'موسيقي / استعراضي', 'ﺧﻴﺎﻝ', 'ﺭﻋﺐ', 'ﺭﻭﻣﺎﻧﺴﻲ', 'ﺳﻴﺮﺓ ﺫاﺗﻴﺔ'}

####  Training DM Model

In [435]:
model_dm = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dm.build_vocab([document for document in tqdm(train_tagged.values)])

100%|██████████| 829/829 [00:00<00:00, 927963.17it/s]


In [436]:
%%time
for epoch in range(30):
    model_dm.train(utils.shuffle([document for document in \
                tqdm(train_tagged.values)]),\
                total_examples=len(train_tagged.values), epochs=1)
    model_dm.alpha -= 0.002
    model_dm.min_alpha = model_dm.alpha

100%|██████████| 829/829 [00:00<00:00, 2082082.64it/s]


CPU times: user 5.3 s, sys: 448 ms, total: 5.75 s
Wall time: 3.88 s


In [439]:
y_train, X_train = fetch_targets_regressors(model_dm, train_tagged)
y_test, X_test = fetch_targets_regressors(model_dm, test_tagged)


SGD_classifier = SGDClassifier(loss="hinge", penalty="l2", max_iter=50)
SGD_classifier.fit(X_train, y_train)
y_predict = SGD_classifier.predict(X_test)

print('Testing accuracy %s' % accuracy_score(y_test, y_predict))
print('Testing F1 score: {}'.format(f1_score(y_test, y_predict,\
                                             average='weighted')))

Testing accuracy 0.985524728588661
Testing F1 score: 0.9848792820181294


Which genres were never predicted?

In [440]:
set(y_test) - set(y_predict)

{'ﺭﻋﺐ'}

#### Predicting Missing Genres Using DM Model

In [450]:
targets = targets.reset_index(drop = True)

In [451]:
targets_docs = targets.apply(
    lambda row: TaggedDocument(words=row['ملخص'], tags=row['التصنيف']), axis=1)

In [464]:
genres, synopsis = \
    fetch_targets_regressors(model_dm, targets_docs)
predicted_genres = \
SGD_classifier.predict(synopsis)

In [475]:
targets['التصنيف'] = predicted_genres

#### ReMerge Divided Dataframes Again

In [483]:
movies = pd.concat([targets, short_movies, movies_dataset])

In [487]:
get_null_percentage(movies)

>> ‫اسم الفيلم: 0%
>> ‫الشهر: 0%
>> ‫السنة: 0%
>> ‫التصنيف: 0%
>> ‫التقييم: 0%
>> ‫المدة: 0%
>> ‫ملخص: 0%
>> ‫ممثلات: 0%
>> ‫ممثلون: 0%
>> ‫تأليف: 3%
>> ‫إخراج: 2%


## Questions to Ask

### Which movie genre dominated which decade?

In [ ]:
genre_names= [genre_added_40s['التصنيف'].value_counts().idxmax()\
        ,genres_added_50s['التصنيف'].value_counts().idxmax()
        ,genre_added_60s['التصنيف'].value_counts().idxmax()
        ,genre_added_70s['التصنيف'].value_counts().idxmax()]

genre_max_counts = [genre_added_40s['التصنيف'].value_counts()[0] \
        ,genres_added_50s['التصنيف'].value_counts().head()[0] \
        ,genre_added_60s['التصنيف'].value_counts().head()[0] \
        ,genre_added_70s['التصنيف'].value_counts().head()[0]]

In [ ]:
print('40s: ' + genre_names[0] + ' with ' + str(genre_max_counts[0]) + '  movies')
print('50s: ' + genre_names[1] + ' with ' + str(genre_max_counts[1]) +  ' movies')
print('60s: ' + genre_names[2] + ' with ' + str(genre_max_counts[2]) + ' movies')
print('70s: ' + genre_names[3] + ' with ' + str(genre_max_counts[3]) + ' movies')

### Which movie genre had the least existence which decade?

In [ ]:
genre_names= [genre_added_40s['التصنيف'].value_counts().idxmin()\
        ,genres_added_50s['التصنيف'].value_counts().idxmin()
        ,genre_added_60s['التصنيف'].value_counts().idxmin()
        ,genre_added_70s['التصنيف'].value_counts().idxmin()]

genre_min_counts = [genre_added_40s['التصنيف'].value_counts().min() \
        ,genres_added_50s['التصنيف'].value_counts().min() \
        ,genre_added_60s['التصنيف'].value_counts().min() \
        ,genre_added_70s['التصنيف'].value_counts().min()]


In [ ]:
print('40s: ' + genre_names[0] + ' with ' + str(genre_min_counts[0]) + '  movies')
print('50s: ' + genre_names[1] + ' with ' + str(genre_min_counts[1]) +  ' movies')
print('60s: ' + genre_names[2] + ' with ' + str(genre_min_counts[2]) + ' movies')
print('70s: ' + genre_names[3] + ' with ' + str(genre_min_counts[3]) + ' movies')

### Which is the highest rated films throughout the 4 decades?

In [ ]:
highest_rated_movie = movies_df.loc[movies_df['التقييم'].idxmax()]
print('Movie Name: ' + highest_rated_movie.values[0])
print('Movie Rating: ' + str(highest_rated_movie.values[3]))
print('Movie Year: ' + highest_rated_movie.values[2])
print('Movie Genre: ' + highest_rated_movie.values[4])
print('Movies Actors: ' + highest_rated_movie.values[8])
print('Movies Authors:  ' + highest_rated_movie.values[7])

### What is the average duration of each genre?

In [ ]:
print('40s decade')
print(ratings_added_50s.groupby(['التصنيف'])['المدة'].mean())
print('-----------------')
print('50s decade')
print(ratings_added_60s.groupby(['التصنيف'])['المدة'].mean())
print('-----------------')
print('60s decade')
print(ratings_added_60s.groupby(['التصنيف'])['المدة'].mean())
print('70s decade')
print('-----------------')
print(ratings_added_70s.groupby(['التصنيف'])['المدة'].mean())

### Which Director directed the most movies per decade?

In [ ]:
print(movies_df['إخراج'].value_counts().head())

In [ ]:
ratings_added_40s.groupby('التصنيف')['التقييم'].mean().plot.barh()


### What are the average ratings with respect to genres?

In [ ]:
print('40s decade')
print(ratings_added_50s.groupby(['التصنيف'])['المدة'].mean())
print('-----------------')
print('50s decade')
print(ratings_added_60s.groupby(['التصنيف'])['المدة'].mean())
print('-----------------')
print('60s decade')
print(ratings_added_60s.groupby(['التصنيف'])['المدة'].mean())
print('70s decade')
print('-----------------')
print(ratings_added_70s.groupby(['التصنيف'])['المدة'].mean())